<a href="https://colab.research.google.com/github/jerryjliu/llama_index/blob/main/docs/docs/examples/evaluation/mt_bench_human_judgement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="在 Colab 中打开"/></a>


# 在MT-Bench人工判断的`LabelledPairwiseEvaluatorDataset`上对LLM评估器进行基准测试


在这个笔记本指南中，我们使用稍微调整过的MT-Bench人类评判数据集对Gemini和GPT模型进行基准测试，作为LLM评估器。对于这个数据集，人类评估者会比较两个LLM模型对于给定查询的响应，并根据自己的偏好对它们进行排名。在原始版本中，对于给定的示例（查询，两个模型的响应），可能会有多个人类评估者。然而，在我们考虑的调整版本中，我们会聚合这些“重复”的条目，并将原始模式的“获胜者”列转换为代表“model_a”在所有人类评估者中获胜比例的形式。为了适应llama数据集，并更好地考虑平局（尽管样本较小），我们为这个比例设定了一个不确定性阈值，即如果它在[0.4, 0.6]之间，我们认为两个模型之间没有获胜者。我们从[llama-hub](https://llamahub.ai)下载这个数据集。最后，我们要对以下LLM进行基准测试：

1. GPT-3.5（OpenAI）
2. GPT-4（OpenAI）
3. Gemini-Pro（Google）


In [ ]:
%pip install llama-index-llms-openai
%pip install llama-index-llms-cohere
%pip install llama-index-llms-gemini

In [ ]:
!pip install "google-generativeai" -q

In [ ]:
import nest_asyncio

nest_asyncio.apply()

### 加载数据集

让我们从llama-hub加载llama数据集。


In [ ]:
from llama_index.core.llama_dataset import download_llama_dataset# 下载数据集pairwise_evaluator_dataset, _ = download_llama_dataset(    "MtBenchHumanJudgementDataset", "./mt_bench_data")

In [ ]:
pairwise_evaluator_dataset.to_pandas()[:5]

,query,answer,second_answer,contexts,ground_truth_answer,query_by,answer_by,second_answer_by,ground_truth_answer_by,reference_feedback,reference_score,reference_evaluation_by
0,Compose an engaging travel blog post about a r...,I recently had the pleasure of visiting Hawaii...,Aloha! I recently had the pleasure of embarkin...,None,None,human,ai (alpaca-13b),ai (gpt-3.5-turbo),None,None,0.0,human
1,Compose an engaging travel blog post about a r...,I recently had the pleasure of visiting Hawaii...,Aloha and welcome to my travel blog post about...,None,None,human,ai (alpaca-13b),ai (vicuna-13b-v1.2),None,None,0.0,human
2,Compose an engaging travel blog post about a r...,Here is a draft travel blog post about a recen...,I recently had the pleasure of visiting Hawaii...,None,None,human,ai (claude-v1),ai (alpaca-13b),None,None,1.0,human
3,Compose an engaging travel blog post about a r...,Here is a draft travel blog post about a recen...,Here is a travel blog post about a recent trip...,None,None,human,ai (claude-v1),ai (llama-13b),None,None,1.0,human
4,Compose an engaging travel blog post about a r...,Aloha! I recently had the pleasure of embarkin...,I recently had the pleasure of visiting Hawaii...,None,None,human,ai (gpt-3.5-turbo),ai (alpaca-13b),None,None,1.0,human


### 定义我们的评估器


In [ ]:
from llama_index.core.evaluation import PairwiseComparisonEvaluator
from llama_index.llms.openai import OpenAI
from llama_index.llms.gemini import Gemini
from llama_index.llms.cohere import Cohere


llm_gpt4 = OpenAI(temperature=0, model="gpt-4")
llm_gpt35 = OpenAI(temperature=0, model="gpt-3.5-turbo")
llm_gemini = Gemini(model="models/gemini-pro", temperature=0)

evaluators = {
    "gpt-4": PairwiseComparisonEvaluator(llm=llm_gpt4),
    "gpt-3.5": PairwiseComparisonEvaluator(llm=llm_gpt35),
    "gemini-pro": PairwiseComparisonEvaluator(llm=llm_gemini),
}

### 使用`EvaluatorBenchmarkerPack`（llama-pack）进行基准测试

为了比较我们的四个评估器，我们将它们与`MTBenchHumanJudgementDataset`进行基准测试，其中参考由人类评估者提供。基准测试将返回以下数量：

- `number_examples`：数据集包含的示例数量。
- `invalid_predictions`：无法产生最终评估的评估数量（例如，由于无法解析评估输出或LLM评估器抛出异常）。
- `inconclusives`：由于这是一种成对比较，为了减少“位置偏见”的风险，我们进行两次评估 —— 一次是按照呈现两个模型答案的原始顺序进行，另一次是按照呈现给评估器LLM的顺序进行翻转。如果LLM评估器在第二个顺序中翻转其投票以与第一次投票相比，则结果是无法确定的。
- `ties`：`PairwiseComparisonEvaluator`也可以返回“平局”结果。这是给出平局结果的示例数量。
- `agreement_rate_with_ties`：LLM评估器在包括平局时与参考（在本例中是人类）评估者达成一致的比率。用于计算此指标的分母为：`number_examples - invalid_predictions - inconclusives`。
- `agreement_rate_without_ties`：LLM评估器在不包括平局时与参考（在本例中是人类）评估者达成一致的比率。用于计算此指标的分母为：`number_examples - invalid_predictions - inconclusives - ties`。

为了计算这些指标，我们将使用`EvaluatorBenchmarkerPack`。


In [ ]:
from llama_index.core.llama_pack import download_llama_pack

EvaluatorBenchmarkerPack = download_llama_pack(
    "EvaluatorBenchmarkerPack", "./pack"
)

GPT-3.5 是 OpenAI 推出的一种自然语言处理模型，它是 GPT-3 的改进版本。这个模型具有更强大的语言理解和生成能力，可以用于各种文本生成任务，如对话系统、文章写作和翻译等。 GPT-3.5 使用了大量的预训练数据和深度学习技术，使其在处理自然语言任务时表现出色。


In [ ]:
evaluator_benchmarker = EvaluatorBenchmarkerPack(
    evaluator=evaluators["gpt-3.5"],
    eval_dataset=pairwise_evaluator_dataset,
    show_progress=True,
)

In [ ]:
gpt_3p5_benchmark_df = await evaluator_benchmarker.arun(
    batch_size=100, sleep_time_in_seconds=0
)

In [ ]:
gpt_3p5_benchmark_df.index = ["gpt-3.5"]
gpt_3p5_benchmark_df

,number_examples,invalid_predictions,inconclusives,ties,agreement_rate_with_ties,agreement_rate_without_ties
gpt-3.5,1204,82,393,56,0.736626,0.793462


GPT-4

GPT-4是OpenAI公司开发的第四代通用预训练模型。它是一种基于人工智能的语言模型，可以用于生成文本、回答问题和执行其他自然语言处理任务。GPT-4相较于之前的版本在语言理解和生成方面有了显著的改进，能够更准确地理解和生成自然语言。


In [ ]:
evaluator_benchmarker = EvaluatorBenchmarkerPack(
    evaluator=evaluators["gpt-4"],
    eval_dataset=pairwise_evaluator_dataset,
    show_progress=True,
)

In [ ]:
gpt_4_benchmark_df = await evaluator_benchmarker.arun(
    batch_size=100, sleep_time_in_seconds=0
)

In [ ]:
gpt_4_benchmark_df.index = ["gpt-4"]
gpt_4_benchmark_df

,number_examples,invalid_predictions,inconclusives,ties,agreement_rate_with_ties,agreement_rate_without_ties
gpt-4,1204,0,100,103,0.701087,0.77023


### Gemini Pro

注意：Gemini模型的速率限制仍然非常严格，这是可以理解的，因为在撰写本笔记时它们刚刚发布。因此，我们使用非常小的`batch_size`和适度高的`sleep_time_in_seconds`来降低被限制速率的风险。


In [ ]:
evaluator_benchmarker = EvaluatorBenchmarkerPack(
    evaluator=evaluators["gemini-pro"],
    eval_dataset=pairwise_evaluator_dataset,
    show_progress=True,
)

In [ ]:
gemini_pro_benchmark_df = await evaluator_benchmarker.arun(
    batch_size=5, sleep_time_in_seconds=0.5
)

In [ ]:
gemini_pro_benchmark_df.index = ["gemini-pro"]
gemini_pro_benchmark_df

,number_examples,invalid_predictions,inconclusives,ties,agreement_rate_with_ties,agreement_rate_without_ties
gemini-pro,1204,2,295,60,0.742007,0.793388


In [ ]:
evaluator_benchmarker.prediction_dataset.save_json("gemini_predictions.json")

### 概要

为了方便起见，让我们将所有的结果放在一个单独的DataFrame中。


In [ ]:
import pandas as pd

final_benchmark = pd.concat(
    [
        gpt_3p5_benchmark_df,
        gpt_4_benchmark_df,
        gemini_pro_benchmark_df,
    ],
    axis=0,
)
final_benchmark

,number_examples,invalid_predictions,inconclusives,ties,agreement_rate_with_ties,agreement_rate_without_ties
gpt-3.5,1204,82,393,56,0.736626,0.793462
gpt-4,1204,0,100,103,0.701087,0.770230
gemini-pro,1204,2,295,60,0.742007,0.793388


从上面的结果中，我们得出以下观察结果：
- 就一致性率而言，三个模型似乎非常接近，或许Gemini模型稍微领先一些
- Gemini Pro和GPT-3.5似乎比GPT-4更加肯定，导致只有50-60次与GPT-4的100次并列。
- 然而，也许与前一点有关，GPT-4产生的无法确定的情况最少，这意味着它在位置偏见方面受到的影响最小。
- 总的来说，看起来Gemini Pro与GPT模型不相上下，并且可以说它的表现优于GPT-3.5 — 看起来Gemini可以成为评估任务中GPT模型的合法替代品。
